# Latvia

Open zip file, go to folder projekcijas, then open a00000031.gdbtable in QGIS

In [55]:
import pandas as pd
import geopandas as gpd
import requests
from bs4 import BeautifulSoup
import xml.etree.ElementTree as ET

pd.options.mode.chained_assignment = None

## Fetch results

In [56]:
url = 'http://ep2014.cvk.lv/activities.xml'

In [57]:
root = ET.fromstring(requests.get(url).text)
regions = root.findall('region')
regions = [{'name': x.get('name'), 'turnout': x.find('total').find('value').text} for x in regions]

In [60]:
df = pd.DataFrame(regions)

In [71]:
df.to_csv('../../data/raw/lva.csv', index=False)

## Merge

In [58]:
geo = gpd.read_file('../../data/shapefiles/lva/lva.shp')

In [62]:
geo = geo.to_crs(epsg=4326)

In [63]:
geo = geo.merge(df, left_on='Nos_pilns', right_on='name', how='left')

In [64]:
geo.turnout = geo.turnout.astype(float)

## Export

In [68]:
geo = geo[['name', 'turnout', 'geometry']]
geo['country'] = 'lva'

In [69]:
geo.to_file('../../data/processed/lva')